In [2]:
from datetime import date
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium import webdriver # driving the actions, links up with the browser
from bs4 import BeautifulSoup
import time
import re
from pick import pick
import progressbar
import os

In [12]:
# change the path

search_webpage = 'https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm'
PATH = "D:\\Program Files (x86)\\chromedriver.exe"
#PATH =  r'C:\Users\esthe\Documents\Coding\chromedriver_win32\chromedriver.exe'
#driver = webdriver.Chrome(PATH)
#driver.get(search_webpage)

In [4]:
yes = ["Yes", "Y", "YES", "y", "yes"]
starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
dashboard_url = "http://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
login_url = 'https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm'
output_name = 'WW_postings.csv'
choice = 'Application Deadlines in the next 10 Days'
potential_html1 = None
potential_html2 = None
potential_html3 = None

In [5]:
{"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 

{'Work Term Duration:': None,
 'Job Summary:': None,
 'Job Responsibilities:': None,
 'Required Skills:': None,
 'Targeted Degrees and Disciplines:': None,
 'Application Documents Required:': None}

In [6]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [1]:
df

NameError: name 'df' is not defined

In [13]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])

def main():
    starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"

    output_name = 'WW_postings.csv'
    
    browser = webdriver.Chrome(PATH)
    browser.get(starting_page) # in original script, this was starting page

    input('Are you ready?')

    
    data = get_job_lists(choice, browser, output_name)

    print("Done!")


def get_job_lists(choice, browser, output_name):
    
    print("Getting job lists ...")

    # Navigate to the job listings page then wait for the javascript to load
    browser.find_element_by_link_text(choice).click()

    input("Are you ready again?")
    
    # Get the HTML of the page and check if this is the last page (next_page_buttons)
    page_html = browser.execute_script("return document.body.innerHTML")
    pattern = r'<a href=".+?" onclick="loadPostingTable(.+?)">\s*»\s*<\/a>'
    next_page_buttons = []
    next_page_buttons = re.findall(pattern, page_html)



    # bar = progressbar.ProgressBar(max_value=self.job_lists_page_count)
    # bar.update(self.gather_current_progress)


    # dots for loading screen
    dots = ""
    counter_page = 0
    # Scrape the tables and save it to output_name as a CSV
    job_posting_importants_order = ['Work Term Duration:', "Job Summary:", "Job Responsibilities:", "Required Skills:", 
                                    "Targeted Degrees and Disciplines:", "Application Documents Required:"]
    with open(output_name, 'w') as f:
        row_count = 0
        job_count = 0
        while (counter_page < 1):
            counter_page += 1
            # loading screen stuff
            os.system('clear')
            print("Working" + dots)
            # update dots
            if (dots == "..."):
                dots = ""
            else:
                dots += "."

            # wait for JavaScript to load then download the page info
            time.sleep(2)
            soup = BeautifulSoup(page_html, "html.parser")
            
            for tr in soup.find_all('tr')[2:]: # iterating throw all rows
            
                
                job_count += 1
                print(job_count)
                try:
                    # all the data that was not in the original script
                    job_posting_importants = {"Work Term Duration:": None, # might need to do some regex filtering
                                             "Job Summary:" : None,
                                             "Job Responsibilities:": None,
                                             "Required Skills:" : None,
                                             "Targeted Degrees and Disciplines:": None,
                                             "Application Documents Required:": None} 


                    tds = tr.find_all('td') # iterating all cells in each row.
                    count = 0
                    job_info_list = []
                    for x in tds[0:-1]: # starting with the 3 buttons


                        if (count == 0): # at 3 buttons
                            link_container = x.find_all('a')[2]

                            onclick_link = link_container['onclick']

                            # clicking the onclick link:
                            browser.execute_script(onclick_link)
                            browser.switch_to.window(browser.window_handles[1])

                            # give the browser 1 seconds for the website to catch up with the code
                            browser.implicitly_wait(2)

                            time.sleep(3)
                            # do stuff, locate the texts of 'job posting importants'

                            #saving the contents of the html of the new page:
                            indiv_html = browser.execute_script("return document.body.innerHTML")
                            indiv_posting = BeautifulSoup(indiv_html, "html.parser")

                            # finding the third 'table' in the page, as the first table is no use to us

                            job_posting_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[1]


                            
                            #print(job_posting_information_table)


                            # iterating through all rows in the table
                            for row in job_posting_information_table.find_all('tr'): 
                                cell_list = row.find_all('td')
                                try:
                                    key = cell_list[0].get_text().strip()
                                except:
                                    continue


                                if (key in job_posting_importants):
                                    try:
                                        job_posting_importants[key] = cell_list[1].get_text().strip()
                                    except:
                                        job_posting_importants[key] = ""


                            application_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[2]
                            job_posting_importants["Application Documents Required:"] = application_information_table.find_all('td')[3].get_text().strip()

                            browser.close() # close current tab
                            browser.switch_to.window(browser.window_handles[0])

                            #print("lala")
                        if (count >= 2):  
                            #print(x.text)
                            job_info_list.append(str(x.text))

                        count += 1


                        
                    for col in job_posting_importants_order:
                        job_info_list.append(job_posting_importants[col])

                    df.loc[row_count] = job_info_list
                    row_count+=1
                except:
                    continue
                
            # navigate to the next page and get the HTML, next page buttons
            next_page_buttons = re.findall(pattern, page_html)
            browser.find_element_by_link_text("»").click()
            time.sleep(2)
            page_html = browser.execute_script("return document.body.innerHTML")


if __name__ == "__main__":
    main()

Are you ready?a
Getting job lists ...
Are you ready again?a
Working
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Done!


In [26]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185577,\n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Planner\n\...,City of St Catharines,Municipal Office,1,Open for Applications,St. Catharines,"Intermediate, Senior",12,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Student Planner,To research assigned projects for data relevan...,Students should be experienced in virtual meet...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,186774,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,City of Waterloo,Development Services,1,Open for Applications,Waterloo,"Intermediate, Senior",63,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,The successful candidate will have involvement...,Growth Management Projects may include:\n\nLan...,The candidate should have a strong working kno...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,185901,\n\n\n\n\t\t\t\t\t\t\t\t\t\tFleet Assistant\n\...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,9,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,A co-op opportunity exists for a student to su...,"Provide support to the Fleet Coordinator, Flee...",Currently enrolled in a relevant post-secondar...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
3,186759,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,17,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,A co-op position exists for an individual to a...,Assist with park bookings and event scheduling...,1-2 years of related post-secondary education ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
4,185951,\n\n\n\n\t\t\t\t\t\t\t\t\t\tCommunity Developm...,City of Windsor,Divisional Office,1,Open for Applications,Windsor,"Junior, Intermediate, Senior",18,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,We are looking for an eager and dedicated indi...,Under the supervision and direction of the Man...,The successful applicant will have an excellen...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,184464,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst, eDiscover...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",7,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Learn from deep subject matter experts through...,About the team \n \nWith the proliferation of ...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
96,184476,\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst Intern - I...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Partner with clients to solve their most compl...,About the team \nThe Innovation & Technology C...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
97,184485,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst, Financial...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",23,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Learn from deep subject matter experts through...,About the team \n \nThe Financial Engineering ...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"Un

In [27]:
df.to_csv('page6.csv')

In [68]:
#df = pd.read_csv('half_data.csv')
df2 = pd.read_csv('page6.csv')


In [73]:
df2


,Unnamed: 0,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185577,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of St Catharines,Municipal Office,1,Open for Applications,St. Catharines,"Intermediate, Senior",12,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,1,186774,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Development Services,1,Open for Applications,Waterloo,"Intermediate, Senior",63,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,2,185901,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,9,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
3,3,186759,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Waterloo,Divisional Office,1,Open for Applications,Waterloo,Junior,17,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
4,4,185951,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,City of Windsor,Divisional Office,1,Open for Applications,Windsor,"Junior, Intermediate, Senior",18,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,184464,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",7,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
96,96,184476,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",20,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
97,97,184485,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",23,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
98,98,184489,0 \n\n\n\n\t\t\t\t\t\t\t\t\t\tStudent Plan...,Deloitte,Tax and Audit,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",11,"0 \n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t...",4 month work term,0 Studen...,0 To research assigned projects for data r...,0 Students should be experienced in virtua...,0 0 Targeted Clusters\n\t\t\t\n\n\t\t\...,"University of Waterloo Co-op W

In [ ]:
a = {"bob": 2, 'sam':6, 'asd':123}
a.values()

In [ ]:
' '.join("j, i, s".split())

In [ ]:
e2jin@uwaterloo.ca
Eevee1218.

In [5]:
'''
Things to Change
0. Get rid of whitespaces for job title - DONE
1. Get rid of capital NEW at the start of job titles - DONE
2. get rid of divison - DONE 
3. get rid of internal status - DONE
4. turn level into 3 binary columns - Dropped it, don't think its that important
5. filter all until only app deadline <= jan 19 - DONE
6. whitespace for targeted clusters - DONE
7. App docs required turns into need cover letter? - DONE
8. change Work Term Duration to [1,4]
9. whitepsace for app deadlinke - DONE

'''
df1 = pd.read_csv('page46.csv')
df2 = pd.read_csv('page17_19.csv')
df3 = pd.read_csv('page20_22.csv')
df5 = pd.read_csv('page23_25.csv')
df6 = pd.read_csv('page26_27.csv')

In [6]:
df0 = pd.read_excel("page13.xlsx")
df7 = pd.read_excel("page_11_13.xlsx")
df8 = pd.read_excel("page_7_10.xlsx")
df9 = pd.read_excel("page_14_17.xlsx")
frames = [df0, df7, df8, df9,df1, df2, df3, df5, df6]

df = pd.concat(frames)
df.drop(["Unnamed: 0"], axis=1, inplace = True)
df = df.fillna("")
df.reset_index(inplace=True)

In [7]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,\n\n\n\t\t\t\t\t\t\t\t\t\tAccounting Analyst\n...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
1,1,185136,\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.\...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
2,2,186638,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
3,3,186639,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
4,4,186640,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: \n\nConfidence: A...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,\r\n\r\n\t\t\t\t\t\t\t\t\t\tNEW\r\n\t\t\t\t\t\...,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,"\r\n\t\t\t\t\t\t\t\tJan 19, 2021\r\n\t\t\t\t\t...",4 month work term,"At Zoocasa, our mission is to ensure our clien...",Responsibilities:\r\n\r\n- Perform manual test...,Required Skills:\r\n- Experience with manual t...,Targeted Clusters\r\n\t\t\t\r\n\r\n\t\t\r\n\t\...,Grade Report
2511,134,185591,\r\n\r\n\r\n\r\n\t\t\t\t\t\t\t\t\t\tProtein En...,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,"\r\n\t\t\t\t\t\t\t\tJan 19, 2021\r\n\t\t\t\t\t...",8 month consecutive work term required,About the Position\r\nWe are currently seeking...,Your Job \r\n\r\nSupports recombinant protein ...,"Your Academics\r\nWe want you to succeed, so y...",Targeted Clusters\r\n\t\t\t\r\n\r\n\t\t\r\n\t\...,"University of Waterloo Co-op Work History,Cove..."
2512,135,185600,\r\n\r\n\r\n\r\n\t\t\t\t\t\t\t\t\t\tComputatio...,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,"\r\n\t\t\t\t\t\t\t\tJan 19, 2021\r\n\t\t\t\t\t...",8 month consecutive work term required,About the Position\r\n \r\nWe are currently se...,Your Job \r\n \r\n\r\nAssists with the applica...,"Your Academics\r\n \r\nW

In [153]:
import time
from datetime import datetime

In [8]:

def surrounding_space_remover(str):
    return ' '.join(str.split())
    
df_space_remover = np.vectorize(surrounding_space_remover)

In [9]:
tic = time.time()

for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    print(column)
    df[column] = df_space_remover(df[column])
    
toc = time.time()

print(str(1000*(toc-tic)))
    

Job Title
App Deadline
Targeted Degrees and Disciplines
Job Summary
Job Responsibilities
Required Skills
Targeted Degrees and Disciplines
654.7608375549316


In [10]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,"University of Waterloo Co-op Work History,Résu..."
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
2,2,186638,NEW QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,"University of Waterloo Co-op Work History,Résu..."
3,3,186639,NEW Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"Jan 19, 2021",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,4,186640,NEW 1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,NEW QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,"Jan 19, 2021",4 month work term,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,Grade Report
2511,134,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,"University of Waterloo Co-op Work History,Cove..."
2512,135,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Cove..."
2513,136,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,"Jan 19, 2021",8 month consecutive work term preferred,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,"Univers

In [181]:

def surrounding_space_remover(str):
    return ' '.join(str.split())
    
df_space_remover = np.vectorize(surrounding_space_remover)

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185132,\n\n\n\t\t\t\t\t\t\t\t\t\tAccounting Analyst\n...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
1,185136,\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.\...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
2,186638,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
3,186639,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
4,186640,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: \n\nConfidence: A...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,186048,\n\n\n\n\t\t\t\t\t\t\t\t\t\tTechnical Project ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",39,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,POSITION SUMMARY\n\nThe successful candidate w...,RESPONSIBILITIES:\n\nManaging development team...,SKILLS AND QUALIFICATIONS:\n\nCurrently enroll...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
296,186050,\n\n\n\n\t\t\t\t\t\t\t\t\t\tReporting Automati...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Intermediate, Senior",4,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"POSITION SUMMARY\nAs a student at BlackBerry, ...",RESPONSIBILITIES\nResponsibilities include the...,SKILLS AND QUALIFICATIONS\n\nCurrently pursuin...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
297,186063,\n\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity Assurance...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",4,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,POSITION SUMMARY\n\nAs a 4-month Security Assu...,RESPONSIBILITIES\n\nExploring new automated te...,SKILLS AND QUALIFICATIONS\n\nCurrently pursing...,Targeted Clusters\n\t\

In [210]:
tic = time.time()
for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    for i in range(len(df[column])):
        df.at[i, column] = surrounding_space_remover(df.at[i,column])
toc = time.time()

print(str(1000*(toc-tic)))
    

646.263599395752


In [211]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,"University of Waterloo Co-op Work History,Résu..."
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
2,2,186638,NEW QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,"University of Waterloo Co-op Work History,Résu..."
3,3,186639,NEW Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"Jan 19, 2021",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,4,186640,NEW 1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,295,186048,Technical Project Coordinator Student - 4 Mont...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",39,"Jan 19, 2021",4 month work term,POSITION SUMMARY The successful candidate will...,RESPONSIBILITIES: Managing development teams t...,SKILLS AND QUALIFICATIONS: Currently enrolled ...,Targeted Clusters - Theme - Business Administr...,"University of Waterloo Co-op Work History,Résu..."
2696,296,186050,Reporting Automation Developer Student - 4 Mon...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Intermediate, Senior",4,"Jan 19, 2021",4 month work term,"POSITION SUMMARY As a student at BlackBerry, y...",RESPONSIBILITIES Responsibilities include the ...,SKILLS AND QUALIFICATIONS Currently pursuing a...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
2697,297,186063,Security Assurance Research Student - 4 Month ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",4,"Jan 19, 2021",4 month work term,POSITION SUMMARY As a 4-month Security Assuran...,RESPONSIBILITIES Exploring new automated techn...,SKILLS AND QUALIFICATIONS Currently pursing a ...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
2698,298,186069,Software Test Development Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Ottawa,"Junior, Intermediate, Senior",9,"Jan 19, 2021",4 month work term,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES The successful candidate's re...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted

In [185]:
column

'Job Title'

In [186]:
df.at[1, "Job Title"]

array(['\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tSummer Housekeeping Staff\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst, Financial Planning & Business Performance\n\t\t\t\t\t\t\t\t\t\n\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tOptical Algorithms Engineering\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tSummer Housekeeping Staff\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\t\n\n\n\t\t\t\t\t\t\t\t\t\tOptical Algorithms Engineering\n\t\t\t\t\t\t\t\t\t\n\n\n',
       '\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst, Financial Planning & Business Performance\n\t\t\t\t\t\t\t\t\t\n\n\n\n',
       '\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t

In [37]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,"University of Waterloo Co-op Work History,Résu..."
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
2,186638,NEW QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,"University of Waterloo Co-op Work History,Résu..."
3,186639,NEW Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"Jan 19, 2021",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,186640,NEW 1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,186915,NEW Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,"University of Waterloo Co-op Work History,Résu..."
296,186918,NEW Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,"University of Waterloo Co-op Work History,Résu..."
297,186921,NEW Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,"University of Waterloo Co-op Work History,Résu..."
298,186923,NEW eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,"Jan 19, 2021",4 month work term,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,"University of Waterloo Co-op 

In [54]:
def new_removal(str):
    try:
        if (str[0:3] == "NEW"): 
            return str[3:]
    except:
        return str # not enough chars in job title
    return str # if no NEW in job title

df_new_remover = np.vectorize(new_removal)

In [55]:
df['Job Title'] = df_new_remover(df["Job Title"])

In [56]:
def day_extractor(str):
    str = str.replace(",", " ")
    str = [int(s) for s in str.split() if s.isdigit()]
    return str[0]

df_day_extractor = np.vectorize(day_extractor)

df['App Deadline'] = df_day_extractor(df["App Deadline"])
df = df[df["App Deadline"] <= 19]


In [13]:
df_copy = df.copy()

In [20]:
df = df_copy.copy()

In [57]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,4 month work term,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2511,134,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2512,135,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2513,136,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,8 month consecutive work term preferred,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [29]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,"University of Waterloo Co-op Work History,Résu..."
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
2,2,186638,NEW QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,"University of Waterloo Co-op Work History,Résu..."
3,3,186639,NEW Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"Jan 19, 2021",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,"University of Waterloo Co-op Work History,Résu..."
4,4,186640,NEW 1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,NEW QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,"Jan 19, 2021",4 month work term,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,Grade Report
2511,134,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,"University of Waterloo Co-op Work History,Cove..."
2512,135,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...","University of Waterloo Co-op Work History,Cove..."
2513,136,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,"Jan 19, 2021",8 month consecutive work term preferred,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,"Univers

In [46]:
def cover_letter_needed(mystr):
    if (type(mystr) is not str): 
        print("OH NO!")
        print(type(mystr))
        print(str)
    return ("Cover Letter" in mystr)

df_cover_letter_needed = np.vectorize(cover_letter_needed)

In [52]:
df["Application Documents Required"] = df_cover_letter_needed(df["Application Documents Required"])

In [53]:
df

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,"Jan 19, 2021",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,2,186638,NEW QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,3,186639,NEW Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,"Jan 19, 2021",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,4,186640,NEW 1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,"Jan 19, 2021",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,NEW QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,"Jan 19, 2021",4 month work term,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2511,134,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2512,135,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,"Jan 19, 2021",8 month consecutive work term required,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2513,136,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,"Jan 19, 2021",8 month consecutive work term preferred,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [58]:
df.rename(columns={"Application Documents Required": "Cover Letter Needed"}, inplace=True)

D:\Downloads\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [59]:
def work_term_duration(str):
    if (str == '4 month work term'): return 1
    if (str == '8 month consecutive work term required'): return 2
    if (str == '8 month consecutive work term preferred'): return 3
    if (str == '2 work term commitment preferred'): return 4
    if (str == '2 work term commitment required'): return 5
    else: return 6 # in case theres smth weird
    
df_work_term_duration = np.vectorize(work_term_duration)

In [60]:
df["Work Term Duration"] = df_work_term_duration(df["Work Term Duration"])

<ipython-input-60-f926741de716>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Work Term Duration"] = df_work_term_duration(df["Work Term Duration"])


In [61]:
df


,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,133,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2511,134,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2512,135,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2513,136,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [9]:
df_710 = pd.read_excel("page_7_10.xlsx").drop(['Unnamed: 0'], axis=1)

In [10]:
df_710

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,184492,\n\n\n\n\t\t\t\t\t\t\t\t\t\tAnalyst Intern - I...,Deloitte,Tax and Audit,1,Open for Applications,NaN,"Junior, Intermediate, Senior",29,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Partner with clients to solve their most compl...,About the team \n \nHaving advised on many of ...,"Enough about us, let's talk about you. \n \nYo...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
1,185906,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst,...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,Junior,20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,"Enjoy flexible, proactive, and practical benef...",What will your typical day look like?\n\nAs pa...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
2,185909,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tFinancial Analyst,...",Deloitte,Tax and Audit,1,Open for Applications,Toronto,Junior,5,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,Learn from deep subject matter experts through...,What will your typical day look like?\n\nThis ...,"Enough about us, let's talk about you\n \nYou ...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
3,186059,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Deloitte,Tax and Audit,10,Open for Applications,NaN,Junior,41,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,The ARA practice is a multi-disciplinary and c...,"Our Actuarial, Rewards & Analytics (ARA) pract...",Working towards a Bachelor's degree from a lea...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
4,187280,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Deloitte,Tax and Audit,7,Open for Applications,Toronto,"Junior, Intermediate",22,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Build a network of colleagues for life.\nBe ex...,What will your typical day look like? \n\nAs p...,"Enough about us, let's talk about you\n\nYou a...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Cove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,184219,\n\n\n\n\t\t\t\t\t\t\t\t\t\tData Analyst Assoc...,Geotab Inc,Oakville,1,Open for Applications,Oakville,"Junior, Intermediate, Senior",23,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,Who we are:\n \nGeotab is a global leader in I...,Duties and Tasks/Essential Functions:\n\n- Col...,Experience/Skills Required: \n- Completing a U...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,Résumé
296,184220,"\n\n\n\n\t\t\t\t\t\t\t\t\t\tData Analyst, Cust...",Geotab Inc,Oakville,1,Open for Applications,Oakville,"Junior, Intermediate, Senior",17,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,Who we are:\n \nGeotab is a global leader in I...,Your Responsibilities:\n\n- Perform data colle...,Your Qualifications:\n\n- Completing a degree ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,Résumé
297,184224,\n\n\n\t\t\t\t\t\t\t\t\t\tDatabase Administrat...,Geotab Inc,Oakville,1,Open for Applications,Oakville,"Junior, Intermediate, Senior",7,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Who we are:\n\nGeotab is a global leader in Io...,Duties and Tasks/Essential Functions:\n\n- Per...,Experience/Skills Required:\n\n- Completing a ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,Résumé
298,184226,\n\n\n\n\t\t\t\t\t\t\t\t\t\tProduct Management...,Geo

In [48]:
esthers_csv = pd.read_csv("esthersfinal.csv").drop(["Unnamed: 0"], axis=1)

In [49]:
esthers_csv

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,186082,Data Scientist Student - 4 Month Summer Term,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75,19,1,POSITION SUMMARY This role is in BlackBerry Ad...,RESPONSIBILITIES Develop novel ML capabilities...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
1,186083,Business Process Analyst Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69,19,1,POSITION SUMMARY Our Business Process Analyst ...,RESPONSIBILITIES · Creating and fostering prof...,SKILLS AND QUALIFICATIONS · Currently enrolled...,Targeted Clusters - Theme - Business Administr...,False
2,186087,Software Development Student - 8 Month Summerr...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3,19,2,"POSITION SUMMARY If you are an ""out of the box...",RESPONSIBILITIES Design and implementation of ...,SKILLS AND QUALIFICATIONS Good knowledge of Ob...,Targeted Clusters - Theme - Computing: Softwar...,False
3,186091,Software Development Student - 4 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40,19,1,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES Work in a cross-functional te...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
4,186095,Security Response Program Manager Student - 4 ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate",23,19,1,POSITION SUMMARY BlackBerry strives to build a...,RESPONSIBILITIES Act as an organizational foca...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Informa...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
1596,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,False
1597,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",False
1598,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,False


In [55]:
df = df.reset_index().drop(["index", "level_0"], axis=1)

In [56]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,186915,Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,False
1196,186918,Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,False
1197,186921,Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,False
1198,186923,eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,False


In [57]:
frames = [esthers_csv, df]
df = pd.concat(frames)

In [58]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,186082,Data Scientist Student - 4 Month Summer Term,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75,19,1,POSITION SUMMARY This role is in BlackBerry Ad...,RESPONSIBILITIES Develop novel ML capabilities...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
1,186083,Business Process Analyst Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69,19,1,POSITION SUMMARY Our Business Process Analyst ...,RESPONSIBILITIES · Creating and fostering prof...,SKILLS AND QUALIFICATIONS · Currently enrolled...,Targeted Clusters - Theme - Business Administr...,False
2,186087,Software Development Student - 8 Month Summerr...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3,19,2,"POSITION SUMMARY If you are an ""out of the box...",RESPONSIBILITIES Design and implementation of ...,SKILLS AND QUALIFICATIONS Good knowledge of Ob...,Targeted Clusters - Theme - Computing: Softwar...,False
3,186091,Software Development Student - 4 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40,19,1,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES Work in a cross-functional te...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
4,186095,Security Response Program Manager Student - 4 ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate",23,19,1,POSITION SUMMARY BlackBerry strives to build a...,RESPONSIBILITIES Act as an organizational foca...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Informa...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,186915,Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,False
1196,186918,Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,False
1197,186921,Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,False
1198,186923,eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,False


In [69]:
df = df.drop(["index"], axis=1)

In [70]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2497,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2498,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2499,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [62]:
df1 = df.duplicated(subset=['ID'])
df1

0       False
1       False
2       False
3       False
4       False
        ...  
2795    False
2796    False
2797    False
2798    False
2799    False
Length: 2800, dtype: bool

In [64]:
df1 = df1[df1 == True]
df1

864     True
865     True
866     True
867     True
868     True
        ... 
1595    True
1596    True
1597    True
1598    True
1599    True
Length: 318, dtype: bool

In [65]:
df.iloc[867]

ID                                                                             186844
Job Title                                           Engineering Assistant - Vancouver
Organization                                                 RDH Building Science Inc
Division                                                                  Head Office
Openings                                                                            1
Internal Status                                                 Open for Applications
City                                                                          Burnaby
Level                                                            Intermediate, Senior
Applications                                                                       10
App Deadline                                                                       19
Work Term Duration                                                                  1
Job Summary                         RDH is an employee

In [71]:
df.drop_duplicates(subset=['ID'], inplace=True)

In [70]:
df = df.reset_index().drop(["index"], axis=1)
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,186082,Data Scientist Student - 4 Month Summer Term,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75,19,1,POSITION SUMMARY This role is in BlackBerry Ad...,RESPONSIBILITIES Develop novel ML capabilities...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
1,186083,Business Process Analyst Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69,19,1,POSITION SUMMARY Our Business Process Analyst ...,RESPONSIBILITIES · Creating and fostering prof...,SKILLS AND QUALIFICATIONS · Currently enrolled...,Targeted Clusters - Theme - Business Administr...,False
2,186087,Software Development Student - 8 Month Summerr...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3,19,2,"POSITION SUMMARY If you are an ""out of the box...",RESPONSIBILITIES Design and implementation of ...,SKILLS AND QUALIFICATIONS Good knowledge of Ob...,Targeted Clusters - Theme - Computing: Softwar...,False
3,186091,Software Development Student - 4 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40,19,1,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES Work in a cross-functional te...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False
4,186095,Security Response Program Manager Student - 4 ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate",23,19,1,POSITION SUMMARY BlackBerry strives to build a...,RESPONSIBILITIES Act as an organizational foca...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Informa...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,186915,Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,False
2478,186918,Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,False
2479,186921,Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,False
2480,186923,eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,False


In [72]:
df["would_shortlist"] = False

In [72]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [74]:
df.iloc[1, "would_shortlist"] = True

ValueError: Can only index by location with a [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array]

In [74]:
df.to_csv("all_jobs.csv")

In [75]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
0,186082,Data Scientist Student - 4 Month Summer Term,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",75,19,1,POSITION SUMMARY This role is in BlackBerry Ad...,RESPONSIBILITIES Develop novel ML capabilities...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False,False
1,186083,Business Process Analyst Student - 4 Month Su...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",69,19,1,POSITION SUMMARY Our Business Process Analyst ...,RESPONSIBILITIES · Creating and fostering prof...,SKILLS AND QUALIFICATIONS · Currently enrolled...,Targeted Clusters - Theme - Business Administr...,False,False
2,186087,Software Development Student - 8 Month Summerr...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",3,19,2,"POSITION SUMMARY If you are an ""out of the box...",RESPONSIBILITIES Design and implementation of ...,SKILLS AND QUALIFICATIONS Good knowledge of Ob...,Targeted Clusters - Theme - Computing: Softwar...,False,False
3,186091,Software Development Student - 4 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Mississauga,"Junior, Intermediate, Senior",40,19,1,POSITION SUMMARY BlackBerry Advanced Technolog...,RESPONSIBILITIES Work in a cross-functional te...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Softwar...,False,False
4,186095,Security Response Program Manager Student - 4 ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate",23,19,1,POSITION SUMMARY BlackBerry strives to build a...,RESPONSIBILITIES Act as an organizational foca...,SKILLS AND QUALIFICATIONS Currently enrolled i...,Targeted Clusters - Theme - Computing: Informa...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,186915,Product Development Co-op,O2 Industries,Product Development,3,Open for Applications,Kitchener,"Junior, Intermediate, Senior",77,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support research a...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Human Factors and ...,False,False
2478,186918,Product Analyst Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",45,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support product ma...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ENG - Manage...,False,False
2479,186921,Finance Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",28,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Support the execut...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters ARTS - Business ARTS - Econo...,False,False
2480,186923,eCommerce Marketing Co-op,O2 Industries,Product Development,1,Open for Applications,Kitchener,"Junior, Intermediate, Senior",20,19,1,"At O2 Industries, we're shaping the future of ...",What You'll Do in This Role Work cross-functio...,What You Bring to This Role Enrollment in a Ba...,Targeted Clusters - Theme - Business Administr...,False,False


In [219]:
esther_shortllist = pd.read_csv("esthershortlist1.csv")

In [220]:
esther_shortllist

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline
0,188759,NEW Python/Django Programmer,4GL Solutions,Divisional Office,1,Open for Applications,Stouffville,Intermediate,14,19-Jan-21
1,187777,NEW Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",59,19-Jan-21
2,183894,Engineering Co-op,ActiveState Software Inc,Divisional Office,2,Open for Applications,Vancouver,"Intermediate, Senior",11,19-Jan-21
3,188750,NEW Software Developer,AdHawk Microsystems,Divisional Office,1,Open for Applications,Kitchener,Senior,24,19-Jan-21
4,187277,NEW Computer Vision Intern (Startup),Akasha Imaging,Divisional Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",106,19-Jan-21
...,...,...,...,...,...,...,...,...,...,...
115,185126,Software Developer,Varicent,Development,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",46,19-Jan-21
116,187337,"NEW Associate Data Engineering - Platform, Inf...",Wattpad,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",29,19-Jan-21
117,187923,NEW Data Engineering,Wonolo Inc,Head Office,1,Open for Applications,Remote in Canada,"Intermediate, Senior",28,19-Jan-21
118,185105,Data Science Associate,XE.com,Divisional Office,1,Open for Applications,Newmarket,"Intermediate, Senior",17,19-Jan-21


In [221]:
bill_shortlist = pd.read_csv("bill_shortlist.csv")

In [222]:
bill_shortlist

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Unnamed: 10
0,187777,NEW Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",59,19-Jan-21,NaN
1,182773,Software Development Engineering - Summer 2021...,Amazon.com Inc,Head Office,1,Open for Applications,NaN,"Intermediate, Senior",312,19-Jan-21,NaN
2,186284,Data Engineering,ApplyBoard,Divisional Office,1,Open for Applications,Kitchener,"Intermediate, Senior",23,19-Jan-21,NaN
3,187165,NEW Data Science Analyst,Baseline Capital,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",27,19-Jan-21,NaN
4,185790,Machine Learning Developer Student - 4 Month S...,BlackBerry,Head Office,4,Open for Applications,Waterloo,"Junior, Intermediate, Senior",85,19-Jan-21,NaN
5,186200,Software Development Student - 8 Month Summer ...,BlackBerry,Head Office,1,Open for Applications,Waterloo,"Junior, Intermediate, Senior",2,19-Jan-21,NaN
6,188141,NEW Data and Analytics Student - Summer 2021 -...,Canadian Tire Corporation Limited,Corporate Office,2,Open for Applications,NaN,"Junior, Intermediate, Senior",43,19-Jan-21,NaN
7,186832,NEW Data Scientist,Capital One,Toronto Office,2,Open for Applications,Toronto,Intermediate,148,19-Jan-21,NaN
8,187358,NEW Data Engineering,Carta Maple Technologies Inc.,Divisional Office,1,Open for Applications,Kitchener,"Intermediate, Senior",33,19-Jan-21,NaN
9,186291,Data Science,Clearbanc,Divisional Office,3,Open for Applications,Toronto,"Intermediate, Senior",27,19-Jan-21,NaN


In [223]:
bill_shortlist.drop(["Unnamed: 10"], axis = 1, inplace = True)

In [224]:
victor_shortlist = pd.read_csv("victor_shortlist.csv")

In [225]:
victor_shortlist.drop(["Unnamed: 10"], axis = 1, inplace = True)

In [226]:
lists = [esther_shortllist, victor_shortlist,bill_shortlist]
sl = pd.concat(lists)

In [227]:
sl.drop_duplicates(subset=['ID'], inplace=True)

In [228]:
sl.shape

(159, 10)

In [229]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True,False
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True,False
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True,False


In [230]:
sl

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline
0,188759,NEW Python/Django Programmer,4GL Solutions,Divisional Office,1,Open for Applications,Stouffville,Intermediate,14,19-Jan-21
1,187777,NEW Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",59,19-Jan-21
2,183894,Engineering Co-op,ActiveState Software Inc,Divisional Office,2,Open for Applications,Vancouver,"Intermediate, Senior",11,19-Jan-21
3,188750,NEW Software Developer,AdHawk Microsystems,Divisional Office,1,Open for Applications,Kitchener,Senior,24,19-Jan-21
4,187277,NEW Computer Vision Intern (Startup),Akasha Imaging,Divisional Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",106,19-Jan-21
...,...,...,...,...,...,...,...,...,...,...
36,185245,NEW Intern - Data Analytics (4 and 8 month),Ontario Teachers' Pension Plan,Toronto Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",64,19-Jan-21
44,187825,NEW Data Science - Corporate Actuarial,The Economical Insurance Group,KW Offices,1,Open for Applications,NaN,Junior,33,19-Jan-21
49,187469,NEW Data Scientist,Validere Technologies,Head Office,1,Open for Applications,Toronto,Junior,40,19-Jan-21
50,186129,Software Developer - 8 months,Visier Inc,Head Office,1,Open for Applications,Vancouver,"Intermediate, Senior",6,19-Jan-21


In [231]:
df["would_shortlist"] = df["ID"].isin(sl["ID"])

In [232]:
def in_shortlist(x):
    if (x in list):
        return True
    else:
        return False
    
df_in_shortlist = np.vectorize(in_shortlist)

In [233]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True,False
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True,False
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True,False


In [238]:
df[df["would_shortlist"] == True]

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
12,188759,Python/Django Programmer,4GL Solutions,Divisional Office,1,Open for Applications,Stouffville,Intermediate,12,19,1,The Company: 4GL Solutions is a software compa...,Your Role: Gain knowledge of our ERP applicati...,What You Bring: Enrolled in Computer Science. ...,Targeted Clusters - Theme - Computing: Softwar...,False,True
25,187777,Data Science,Achievers Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",55,19,3,Data Science Co-Op Toronto Achievers delivers ...,"As a Data Science Co-op, you will: Design and ...",Qualifications: Enrolled in a Data Science or ...,Targeted Clusters - Theme - Computing: Softwar...,False,True
29,183894,Engineering Co-op,ActiveState Software Inc,Divisional Office,2,Open for Applications,Vancouver,"Intermediate, Senior",8,19,1,Engineering Internships (Co-Ops) ActiveState P...,What You'll be Doing As an intern on one of ou...,"Requirements Experience with Linux, Window or ...",Targeted Clusters - Theme - Computing: Informa...,True,True
32,188750,Software Developer,AdHawk Microsystems,Divisional Office,1,Open for Applications,Kitchener,Senior,18,19,1,AdHawk Microsystems is changing the way that h...,AdHawk is looking for software developers to h...,- Strong software development fundamentals - E...,Targeted Clusters - Theme - Computing: Softwar...,False,True
51,187277,Computer Vision Intern (Startup),Akasha Imaging,Divisional Office,2,Open for Applications,Remote in Canada,"Intermediate, Senior",84,19,1,Akasha Imaging (http://akasha.im) is a VC-back...,- Independently implement algorithms for machi...,- Interest in understanding the fundamentals/p...,Targeted Clusters - Theme - Computing: Softwar...,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,187337,"Associate Data Engineering - Platform, Infras...",Wattpad,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",26,19,1,Wattpad is a global multiplatform entertainmen...,- Evolving Wattpad's data infrastructure - Imp...,- Solid understanding of data engineering best...,Targeted Clusters - Theme - Computing: Softwar...,False,True
2407,188451,Data Analyst,William Ashley China Corp (A Fairfax Company),Divisional Office,1,Open for Applications,Vaughan,"Intermediate, Senior",10,19,2,William Ashley William Ashley (WA) was founded...,"As noted above, WA has a pressing need for a n...",Excellent Excel proficiency; be able to write ...,Targeted Clusters - Theme - Computing: Informa...,False,True
2426,187923,Data Engineering,Wonolo Inc,Head Office,1,Open for Applications,Remote in Canada,"Intermediate, Senior",26,19,1,Wonolo (Work Now Locally) is growing and disru...,- Iterate on existing CNN models and in-house ...,- Strong comprehension of statistical analysis...,Targeted Clusters - Theme - Computing: Softwar...,False,True
2448,185105,Data Science Associate,XE.com,Divisional Office,1,Open for Applications,Newmarket,"Intermediate, Senior",17,19,1,The Opportunity This is a unique and exciting ...,Job Responsibilities: You will help build the ...,"Required Knowledge, Skills and Abilities Fluen...","Targeted Clusters - Theme - Data Analytics, St...",True,True


In [236]:
y_data = df["would_shortlist"]

In [237]:
x_data

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True


In [130]:
pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=6310b92290610b58f1a3fc5ab2b0fd76edb9cdb37b1ea25caaafe84731164bb1
  Stored in directory: c:\users\itsbi\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [216]:
x_data.to_csv("x_data.csv")
y_data.to_csv("y_data.csv")

In [218]:
y_data

,would_shortlist
0,False
1,False
2,False
3,False
4,False
...,...
2477,False
2478,False
2479,False
2480,False


In [244]:
x_data = pd.read_csv("x_data.csv").drop(["Unnamed: 0"], axis=1).reset_index(drop= True).fillna("")
y_data = pd.read_csv("y_data.csv").drop(["Unnamed: 0"], axis=1).reset_index(drop= True).fillna("")

In [245]:
y_data

,would_shortlist
0,False
1,False
2,False
3,False
4,False
...,...
2477,False
2478,False
2479,False
2480,False


In [76]:
from sklearn.model_selection import train_test_split

In [180]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=42)

In [182]:
X_train.reset_index(drop= True, inplace=True)
y_train.reset_index(drop= True, inplace=True)

In [246]:
X_train

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one,text_final
0,187891,Finance Assistant,Ubisoft Toronto,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",17,19,1,Our Mission Ubisoft is committed to enriching ...,On being a Finance Assistant Intern at Ubisoft...,What it takes to make it Bachelor's degree in ...,Targeted Clusters - Theme - Accounting and Fin...,True,"[our, mission, ubisoft, is, committed, to, enr...","['mission', 'ubisoft', 'commit', 'enrich', 'pl..."
1,186006,Restorative Care/Recreation Aide,LaPointe-Fisher Nursing Home,Divisional Office,3,Open for Applications,Guelph,"Junior, Intermediate, Senior",13,19,1,LaPointe-Fisher is an accredited 92 bed nursin...,"As a Recreation Aide, you will support the rec...",Organized and self-starter Motivated and posit...,Targeted Clusters - Theme - Health Care: Thera...,True,"[lapointe-fisher, is, an, accredited, 92, bed,...","['accredited', 'bed', 'nursing', 'home', 'loca..."
2,186789,Engineering Intern,Wood Canada Limited,Wood Environment & Infrastructure,2,Open for Applications,Oakville,Intermediate,39,19,1,Wood Environment & Infrastructure Solutions ha...,The successful applicant will work closely wit...,The ideal candidate will have a good understan...,Targeted Clusters - Theme - Construction and I...,False,"[wood, environment, &, infrastructure, solutio...","['wood', 'environment', 'infrastructure', 'sol..."
3,187769,Actuarial Analyst,Brookfield,Brookfield Annuity,1,Open for Applications,Toronto,"Intermediate, Senior",27,19,1,Brookfield Annuity Company is a life insurance...,Pricing group annuities Analyze data and longe...,Prior work term experience in pensions and/or ...,"Targeted Clusters - Theme - Data Analytics, St...",True,"[brookfield, annuity, company, is, a, life, in...","['brookfield', 'annuity', 'company', 'life', '..."
4,187936,Back End Engineering - Core Products,Insticator,Divisional Office,1,Open for Applications,Kitchener,"Intermediate, Senior",6,19,1,About Us: Insticator increases engagement and ...,Work with the Core Products team on applicatio...,Basic Qualifications Solid experience developi...,Targeted Clusters - Theme - Computing: Informa...,False,"[about, us, :, insticator, increases, engageme...","['u', 'insticator', 'increase', 'engagement', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,185573,Technical Assistant,Peto MacCallum Ltd,Kitchener Office,2,Open for Applications,Kitchener,"Junior, Intermediate",39,19,1,"Peto MacCallum Ltd., an independent specialist...",Duties include field sampling and testing of c...,Qualified applicants will be in posession of a...,Targeted Clusters - Theme - Quality Assurance ...,False,"[peto, maccallum, ltd., ,, an, independent, sp...","['peto', 'maccallum', 'independent', 'speciali..."
2229,186228,Vehicle Connected Services,Nissan Canada Inc,Canadian Operations,1,Open for Applications,Mississauga,"Junior, Intermediate",4,19,3,Vehicle Connected Services Co-op Student Job P...,SUMMARY PROJECT DESCRIPTION: Perform testing a...,EDUCATION REQUIREMENTS: Eligible candidates mu...,Targeted Clusters - Theme - Computing: Hardwar...,True,"[vehicle, connected, services, co-op, student,...","['vehicle', 'connect', 'service', 'student', '..."
2230,188913,JK7 TechPubs Writer IP Routing,Nokia,Ottawa Office,1,Open for Applications,Ottawa,"Intermediate, Senior",0,19,1,We create the technology to connect the world....,"Job Responsibilities: Plan, research, write, a...",Qualifications Pursuing a Degree in Engineerin...,Targeted Clusters - Theme - Business Administr...,False,"[we, create, the, technology, to, connect, the...","['create', 'technology', 'connect', 'world', '..."
2231,188416,"Associate, Wood Gundy",Canadian Imperial Bank of Commerce,Wholesale Banking,1,Open for Applications,,"Junior, 

In [241]:
df["3_in_one"] = ""

In [168]:
X_train

,index,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one
2270,2270,187891,Finance Assistant,Ubisoft Toronto,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",17,19,1,Our Mission Ubisoft is committed to enriching ...,On being a Finance Assistant Intern at Ubisoft...,What it takes to make it Bachelor's degree in ...,Targeted Clusters - Theme - Accounting and Fin...,True,
532,532,186006,Restorative Care/Recreation Aide,LaPointe-Fisher Nursing Home,Divisional Office,3,Open for Applications,Guelph,"Junior, Intermediate, Senior",13,19,1,LaPointe-Fisher is an accredited 92 bed nursin...,"As a Recreation Aide, you will support the rec...",Organized and self-starter Motivated and posit...,Targeted Clusters - Theme - Health Care: Thera...,True,
2430,2430,186789,Engineering Intern,Wood Canada Limited,Wood Environment & Infrastructure,2,Open for Applications,Oakville,Intermediate,39,19,1,Wood Environment & Infrastructure Solutions ha...,The successful applicant will work closely wit...,The ideal candidate will have a good understan...,Targeted Clusters - Theme - Construction and I...,False,
1260,1260,187769,Actuarial Analyst,Brookfield,Brookfield Annuity,1,Open for Applications,Toronto,"Intermediate, Senior",27,19,1,Brookfield Annuity Company is a life insurance...,Pricing group annuities Analyze data and longe...,Prior work term experience in pensions and/or ...,"Targeted Clusters - Theme - Data Analytics, St...",True,
435,435,187936,Back End Engineering - Core Products,Insticator,Divisional Office,1,Open for Applications,Kitchener,"Intermediate, Senior",6,19,1,About Us: Insticator increases engagement and ...,Work with the Core Products team on applicatio...,Basic Qualifications Solid experience developi...,Targeted Clusters - Theme - Computing: Informa...,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,1638,185573,Technical Assistant,Peto MacCallum Ltd,Kitchener Office,2,Open for Applications,Kitchener,"Junior, Intermediate",39,19,1,"Peto MacCallum Ltd., an independent specialist...",Duties include field sampling and testing of c...,Qualified applicants will be in posession of a...,Targeted Clusters - Theme - Quality Assurance ...,False,
1095,1095,186228,Vehicle Connected Services,Nissan Canada Inc,Canadian Operations,1,Open for Applications,Mississauga,"Junior, Intermediate",4,19,3,Vehicle Connected Services Co-op Student Job P...,SUMMARY PROJECT DESCRIPTION: Perform testing a...,EDUCATION REQUIREMENTS: Eligible candidates mu...,Targeted Clusters - Theme - Computing: Hardwar...,True,
1130,1130,188913,JK7 TechPubs Writer IP Routing,Nokia,Ottawa Office,1,Open for Applications,Ottawa,"Intermediate, Senior",0,19,1,We create the technology to connect the world....,"Job Responsibilities: Plan, research, write, a...",Qualifications Pursuing a Degree in Engineerin...,Targeted Clusters - Theme - Business Administr...,False,
1294,1294,188416,"Associate, Wood Gundy",Canadian Imperial Bank of Commerce,Wholesale Banking,1,Open for Applications,,"Junior, Intermediate",2,19,1,Job Description IMPORTANT - You must be curren...,How You'll Succeed Client Engagement - Connect...,Who You Are You understand that success is in ...,Targeted Clusters - Theme - Finance and Invest...,True,


In [240]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False,False
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False,False
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False,False
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False,False
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True,False
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True,False
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",4,19,3,About the Position Zymeworks Inc. is proud to ...,Your Job Work on a key software project in the...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Computing: Softwar...,True,False


In [280]:
df["3_in_one_text"] = ""

In [281]:
for i in range(len(df["Work Term Duration"])):
    summary = df["Job Summary"].iloc[i]
    respo = df["Job Responsibilities"].iloc[i]
    skills =  df["Required Skills"].iloc[i]
    df["3_in_one_text"].iloc[i] = summary + respo + skills

D:\Downloads\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [283]:
df["3_in_one_text"]

0       60,000 businesses and millions of people use 1...
1       60,000 businesses and millions of people use 1...
2       1Password is looking for promising new program...
3       We're looking for promising new developers, wh...
4       1Password is looking for promising new program...
                              ...                        
2477    At Zoocasa, our mission is to ensure our clien...
2478    About the Position We are currently seeking a ...
2479    About the Position We are currently seeking a ...
2480    About the Position Zymeworks Inc. is proud to ...
2481    ÀLA.HAUSSE is a peer to peer fashion market di...
Name: 3_in_one_text, Length: 2482, dtype: object

In [393]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, recall_score
np.random.seed(500)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


In [243]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
df["3_in_one"] = [entry.lower() for entry in df["3_in_one"]]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
df["3_in_one"]= [word_tokenize(entry) for entry in df["3_in_one"]]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df["3_in_one"]):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)

In [248]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist,3_in_one,text_final
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False,False,"[60,000, businesses, and, millions, of, people...","['business', 'million', 'people', 'use', 'prot..."
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False,False,"[60,000, businesses, and, millions, of, people...","['business', 'million', 'people', 'use', 'prot..."
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False,False,"[1password, is, looking, for, promising, new, ...","['look', 'promise', 'new', 'programmer', 'join..."
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False,False,"[we, 're, looking, for, promising, new, develo...","['look', 'promise', 'new', 'developer', 'excit..."
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False,False,"[1password, is, looking, for, promising, new, ...","['look', 'promise', 'new', 'programmer', 'join..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False,False,"[at, zoocasa, ,, our, mission, is, to, ensure,...","['zoocasa', 'mission', 'ensure', 'client', 'fi..."
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True,False,"[about, the, position, we, are, currently, see...","['position', 'currently', 'seek', 'student', '..."
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True,False,"[about, the, position, we, are, currently, see...","['position', 'currently', 'seek', 'student', '..."
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior"

In [ ]:
df.to_csv("cleaned_all_data")

In [187]:
X_train.to_csv("3_in_onex_train_cleaned.csv")

In [188]:
X_train

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,3_in_one,text_final
0,187891,Finance Assistant,Ubisoft Toronto,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",17,19,1,Our Mission Ubisoft is committed to enriching ...,On being a Finance Assistant Intern at Ubisoft...,What it takes to make it Bachelor's degree in ...,Targeted Clusters - Theme - Accounting and Fin...,True,"[our, mission, ubisoft, is, committed, to, enr...","['mission', 'ubisoft', 'commit', 'enrich', 'pl..."
1,186006,Restorative Care/Recreation Aide,LaPointe-Fisher Nursing Home,Divisional Office,3,Open for Applications,Guelph,"Junior, Intermediate, Senior",13,19,1,LaPointe-Fisher is an accredited 92 bed nursin...,"As a Recreation Aide, you will support the rec...",Organized and self-starter Motivated and posit...,Targeted Clusters - Theme - Health Care: Thera...,True,"[lapointe-fisher, is, an, accredited, 92, bed,...","['accredited', 'bed', 'nursing', 'home', 'loca..."
2,186789,Engineering Intern,Wood Canada Limited,Wood Environment & Infrastructure,2,Open for Applications,Oakville,Intermediate,39,19,1,Wood Environment & Infrastructure Solutions ha...,The successful applicant will work closely wit...,The ideal candidate will have a good understan...,Targeted Clusters - Theme - Construction and I...,False,"[wood, environment, &, infrastructure, solutio...","['wood', 'environment', 'infrastructure', 'sol..."
3,187769,Actuarial Analyst,Brookfield,Brookfield Annuity,1,Open for Applications,Toronto,"Intermediate, Senior",27,19,1,Brookfield Annuity Company is a life insurance...,Pricing group annuities Analyze data and longe...,Prior work term experience in pensions and/or ...,"Targeted Clusters - Theme - Data Analytics, St...",True,"[brookfield, annuity, company, is, a, life, in...","['brookfield', 'annuity', 'company', 'life', '..."
4,187936,Back End Engineering - Core Products,Insticator,Divisional Office,1,Open for Applications,Kitchener,"Intermediate, Senior",6,19,1,About Us: Insticator increases engagement and ...,Work with the Core Products team on applicatio...,Basic Qualifications Solid experience developi...,Targeted Clusters - Theme - Computing: Informa...,False,"[about, us, :, insticator, increases, engageme...","['u', 'insticator', 'increase', 'engagement', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,185573,Technical Assistant,Peto MacCallum Ltd,Kitchener Office,2,Open for Applications,Kitchener,"Junior, Intermediate",39,19,1,"Peto MacCallum Ltd., an independent specialist...",Duties include field sampling and testing of c...,Qualified applicants will be in posession of a...,Targeted Clusters - Theme - Quality Assurance ...,False,"[peto, maccallum, ltd., ,, an, independent, sp...","['peto', 'maccallum', 'independent', 'speciali..."
2229,186228,Vehicle Connected Services,Nissan Canada Inc,Canadian Operations,1,Open for Applications,Mississauga,"Junior, Intermediate",4,19,3,Vehicle Connected Services Co-op Student Job P...,SUMMARY PROJECT DESCRIPTION: Perform testing a...,EDUCATION REQUIREMENTS: Eligible candidates mu...,Targeted Clusters - Theme - Computing: Hardwar...,True,"[vehicle, connected, services, co-op, student,...","['vehicle', 'connect', 'service', 'student', '..."
2230,188913,JK7 TechPubs Writer IP Routing,Nokia,Ottawa Office,1,Open for Applications,Ottawa,"Intermediate, Senior",0,19,1,We create the technology to connect the world....,"Job Responsibilities: Plan, research, write, a...",Qualifications Pursuing a Degree in Engineerin...,Targeted Clusters - Theme - Business Administr...,False,"[we, create, the, technology, to, connect, the...","['create', 'technology', 'connect', 'world', '..."
2231,188416,"Associate, Wood Gundy",Canadian Imperial Bank of Commerce,Wholesale Banking,1,Open for Applications,,"Junior, 

In [200]:
import ast
word_count = {}
for i in range(len(X_train["Work Term Duration"])):
    for word in ast.literal_eval(X_train["text_final"].iloc[i]):
            word_count[word] = 1
    
        
    

In [199]:
X_train["text_final"].iloc[1]

"['accredited', 'bed', 'nursing', 'home', 'locate', 'guelph', 'ontario', 'operate', 'long', 'term', 'care', 'home', 'providing', 'quality', 'compassionate', 'care', 'service', 'physiotherapy', 'recreation', 'program', 'nutritious', 'food', 'item', 'community', 'look', 'student', 'fill', 'combined', 'role', 'recreation', 'aide', 'restorative', 'recreation', 'aide', 'support', 'recreation', 'department', 'develop', 'run', 'therapeutic', 'program', 'resident', 'one', 'one', 'group', 'basis', 'approxiimately', 'hour', 'biweekly', 'restorative', 'care', 'aide', 'assist', 'resident', 'daily', 'personal', 'need', 'routine', 'main', 'goal', 'maximize', 'resident', 'ability', 'joy', 'living', 'approximately', 'hour', 'biweekly', 'include', 'every', 'weekend', 'organize', 'motivated', 'positive', 'professional', 'attitude', 'ability', 'complete', 'accurate', 'documentation', 'ability', 'understand', 'accept', 'relate', 'people', 'regardless', 'physical', 'psychological', 'emotional', 'state', 'k

In [249]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed,would_shortlist,3_in_one,text_final
0,185132,Accounting Analyst,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",32,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters - Theme - Accounting and Fin...,False,False,"[60,000, businesses, and, millions, of, people...","['business', 'million', 'people', 'use', 'prot..."
1,185136,Security,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",87,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment. ...,Targeted Clusters - Theme - Computing: Softwar...,False,False,"[60,000, businesses, and, millions, of, people...","['business', 'million', 'people', 'use', 'prot..."
2,186638,QA Intern,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",71,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Quality...,False,False,"[1password, is, looking, for, promising, new, ...","['look', 'promise', 'new', 'programmer', 'join..."
3,186639,Junior Developer,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",210,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table: - Confidence: A w...,Targeted Clusters - Theme - Computing: Informa...,False,False,"[we, 're, looking, for, promising, new, develo...","['look', 'promise', 'new', 'developer', 'excit..."
4,186640,1Password for Linux,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",47,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters - Theme - Computing: Softwar...,False,False,"[1password, is, looking, for, promising, new, ...","['look', 'promise', 'new', 'programmer', 'join..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,188060,QA Automation Developer,Zoocasa,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate",12,19,1,"At Zoocasa, our mission is to ensure our clien...",Responsibilities: - Perform manual testing - C...,Required Skills: - Experience with manual test...,Targeted Clusters - Theme - Computing: Softwar...,False,False,"[at, zoocasa, ,, our, mission, is, to, ensure,...","['zoocasa', 'mission', 'ensure', 'client', 'fi..."
2478,185591,Protein Engineering Lab,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate",7,19,2,About the Position We are currently seeking a ...,Your Job Supports recombinant protein producti...,"Your Academics We want you to succeed, so you'...",Targeted Clusters - Theme - Scientific Experim...,True,False,"[about, the, position, we, are, currently, see...","['position', 'currently', 'seek', 'student', '..."
2479,185600,Computational Modelling,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior",3,19,2,About the Position We are currently seeking a ...,Your Job Assists with the application and deve...,"Your Academics We want you to succeed, so you'...","Targeted Clusters - Theme - Data Analytics, St...",True,False,"[about, the, position, we, are, currently, see...","['position', 'currently', 'seek', 'student', '..."
2480,185603,Software Development,Zymeworks Inc.,Divisional Office,1,Open for Applications,Vancouver,"Junior, Intermediate, Senior"

In [250]:
False == 0

True

In [284]:
df["3_in_one_text"]

0       60,000 businesses and millions of people use 1...
1       60,000 businesses and millions of people use 1...
2       1Password is looking for promising new program...
3       We're looking for promising new developers, wh...
4       1Password is looking for promising new program...
                              ...                        
2477    At Zoocasa, our mission is to ensure our clien...
2478    About the Position We are currently seeking a ...
2479    About the Position We are currently seeking a ...
2480    About the Position Zymeworks Inc. is proud to ...
2481    ÀLA.HAUSSE is a peer to peer fashion market di...
Name: 3_in_one_text, Length: 2482, dtype: object

In [285]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['3_in_one_text'],df['would_shortlist'],test_size=0.1)

In [201]:
word_count

{'mission': 1,
 'ubisoft': 1,
 'commit': 1,
 'enrich': 1,
 'player': 1,
 'life': 1,
 'original': 1,
 'memorable': 1,
 'gaming': 1,
 'experience': 1,
 'create': 1,
 'world': 1,
 'people': 1,
 'express': 1,
 'explore': 1,
 'discover': 1,
 'new': 1,
 'possibility': 1,
 'toronto': 1,
 'found': 1,
 'contribute': 1,
 'many': 1,
 'brand': 1,
 'include': 1,
 'assassin': 1,
 'far': 1,
 'honortm': 1,
 'tom': 1,
 'clancy': 1,
 'splinter': 1,
 'watch': 1,
 'team': 1,
 'develop': 1,
 'first': 1,
 'ip': 1,
 'starlink': 1,
 'battle': 1,
 'atlas': 1,
 'lead': 1,
 'development': 1,
 'dog': 1,
 'legion': 1,
 'one': 1,
 'canada': 1,
 'top': 1,
 'employer': 1,
 'young': 1,
 'look': 1,
 'work': 1,
 'enthusiastic': 1,
 'expert': 1,
 'tackle': 1,
 'challenge': 1,
 'entertainment': 1,
 'beyond': 1,
 'offer': 1,
 'open': 1,
 'environment': 1,
 'bright': 1,
 'idea': 1,
 'chance': 1,
 'shine': 1,
 'everyone': 1,
 'eager': 1,
 'share': 1,
 'knowledge': 1,
 'accept': 1,
 'join': 1,
 'u': 1,
 'finance': 1,
 'assist

In [202]:
len(word_count)

14121

In [287]:
Train_X

1613    Hey College/University students! Are you hungr...
1592    Hey College/University students! Are you hungr...
2362    Urbantech is a growing civil engineering consu...
2194    Position Number: Position Title: IT Services, ...
1848    Date: May 2021 ? Aug 2021 (4 Months) Position:...
                              ...                        
845     POSITION SUMMARY In this role, you will have t...
584     WHY LOBLAW? The Loblaw Supply Chain System Tea...
287     POSITION SUMMARY The BlackBerry Open Source Pr...
273     The Field Marketing Intern will support the Bl...
951     Are you looking for unlimited opportunities to...
Name: 3_in_one_text, Length: 2233, dtype: object

In [278]:
df["text_final"]

0       ['business', 'million', 'people', 'use', 'prot...
1       ['business', 'million', 'people', 'use', 'prot...
2       ['look', 'promise', 'new', 'programmer', 'join...
3       ['look', 'promise', 'new', 'developer', 'excit...
4       ['look', 'promise', 'new', 'programmer', 'join...
                              ...                        
2477    ['zoocasa', 'mission', 'ensure', 'client', 'fi...
2478    ['position', 'currently', 'seek', 'student', '...
2479    ['position', 'currently', 'seek', 'student', '...
2480    ['position', 'zymeworks', 'proud', 'offer', 's...
2481    ['peer', 'peer', 'fashion', 'market', 'digital...
Name: text_final, Length: 2482, dtype: object

In [297]:
Tfidf_vect = TfidfVectorizer(max_features=None)
Tfidf_vect.fit(Train_X)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [298]:
print(Tfidf_vect.vocabulary_)

{'hey': 7763, 'college': 3436, 'university': 17314, 'students': 15888, 'are': 1380, 'you': 18332, 'hungry': 8006, 'for': 6803, 'an': 1088, 'exciting': 6170, 'and': 1134, 'challenging': 2995, 'opportunity': 11558, 'in': 8264, 'fast': 6478, 'paced': 11815, 'diverse': 5160, 'organization': 11634, 'if': 8131, 'so': 15265, 'have': 7649, 'future': 7049, 'at': 1587, 'pepsico': 12081, 'as': 1451, 'world': 18211, 'class': 3214, 'we': 17918, 'strive': 15862, 'excellence': 6150, 'everything': 6114, 'that': 16571, 'do': 5195, 'from': 6968, 'new': 11083, 'product': 12904, 'launch': 9382, 'to': 16754, 'delivering': 4694, 'store': 15787, 'accomplish': 557, 'our': 11708, 'goals': 7332, 'need': 10983, 'talented': 16265, 'innovative': 8486, 'people': 12075, 'who': 18043, 'know': 9238, 'how': 7942, 'work': 18167, 'hard': 7611, 'fun': 7007, 'want': 17832, 'make': 9913, 'difference': 4959, 'taste': 16307, 'the': 16572, 'success': 15967, 'with': 18125, 'canada': 2674, 'fastest': 6484, 'growing': 7482, 'cons

In [299]:
Train_X_Tfidf.toarray().shape

(2233, 18393)

In [300]:
print(Train_X_Tfidf)

  (0, 18341)	0.05106685918687452
  (0, 18336)	0.11071946552968719
  (0, 18332)	0.07166297349406603
  (0, 18306)	0.03101466575393565
  (0, 18255)	0.032891441110273915
  (0, 18211)	0.04492309810100032
  (0, 18167)	0.024578078187061817
  (0, 18125)	0.06722433252292581
  (0, 18071)	0.026007151539375436
  (0, 18043)	0.06074286893485016
  (0, 17976)	0.058520134271491275
  (0, 17918)	0.059768480137544495
  (0, 17864)	0.049665607724436435
  (0, 17832)	0.03089422727388261
  (0, 17757)	0.1040854372409525
  (0, 17645)	0.037504197966077894
  (0, 17559)	0.040932032229271866
  (0, 17368)	0.025361168412225115
  (0, 17326)	0.06245069616094547
  (0, 17314)	0.06346791441790683
  (0, 17067)	0.06144818614304881
  (0, 16821)	0.06245069616094547
  (0, 16754)	0.16633932882199706
  (0, 16663)	0.02177299619636236
  (0, 16629)	0.05060558141905136
  :	:
  (2232, 1134)	0.30444044394454783
  (2232, 1088)	0.014556908058647929
  (2232, 1000)	0.05379843824607366
  (2232, 869)	0.05569206560193462
  (2232, 797)	0.04591

In [ ]:
fo

In [307]:
my_alpha = []
for i in range(0, 1000):
    my_alpha += [i / 1000]
    
max_f1_score = float("-inf")
best_alpha = None


    

In [313]:
# fit the training dataset on the NB classifier
for alpha in my_alpha:
    Naive = naive_bayes.MultinomialNB(alpha=alpha)
    Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
    predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
    print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
    current_f1_score = accuracy_score(predictions_NB, Test_Y)*100
    if (current_f1_score > max_f1_score): 
        max_f1_score = current_f1_score
        best_alpha = alpha
    
    

D:\Downloads\lib\site-packages\sklearn\naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  94.77911646586345
Naive Bayes Accuracy Score ->  94.77911646586345
Naive Bayes Accuracy Score ->  94.77911646586345
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy Score ->  94.37751004016064
Naive Bayes Accuracy

Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy

Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy

Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy

Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy

Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy Score ->  95.18072289156626
Naive Bayes Accuracy

In [314]:
max_f1_score

95.18072289156626

In [356]:
import random

In [346]:
import itertools
import random
c_options = [0.2, 1, 4, 8]
kernal_options = ['linear', 'poly', 'rbf', 'sigmoid']
degree_options = [2,3,4] # go through only if poly
gamma_options = ['scale', 'auto'] # run only if not linear
coef0_options = [0,1,10,100] # run only poly sig
probability_options = [1,0]

my_combos = []

a = [c_options, kernal_options, degree_options, gamma_options, coef0_options, probability_options, tol_options]
my_combos = list(itertools.product(*a))


    


In [348]:
len(my_combos)

1680

In [353]:
my_combos

[(0.2, 'linear', 1, 'auto', 0, 1),
 (0.2, 'poly', 2, 'scale', 0, 1),
 (0.2, 'poly', 3, 'scale', 0, 1),
 (0.2, 'poly', 4, 'scale', 0, 1),
 (0.2, 'poly', 2, 'scale', 1, 1),
 (0.2, 'poly', 3, 'scale', 1, 1),
 (0.2, 'poly', 4, 'scale', 1, 1),
 (0.2, 'poly', 2, 'scale', 10, 1),
 (0.2, 'poly', 3, 'scale', 10, 1),
 (0.2, 'poly', 4, 'scale', 10, 1),
 (0.2, 'poly', 2, 'scale', 100, 1),
 (0.2, 'poly', 3, 'scale', 100, 1),
 (0.2, 'poly', 4, 'scale', 100, 1),
 (0.2, 'poly', 2, 'auto', 0, 1),
 (0.2, 'poly', 3, 'auto', 0, 1),
 (0.2, 'poly', 4, 'auto', 0, 1),
 (0.2, 'poly', 2, 'auto', 1, 1),
 (0.2, 'poly', 3, 'auto', 1, 1),
 (0.2, 'poly', 4, 'auto', 1, 1),
 (0.2, 'poly', 2, 'auto', 10, 1),
 (0.2, 'poly', 3, 'auto', 10, 1),
 (0.2, 'poly', 4, 'auto', 10, 1),
 (0.2, 'poly', 2, 'auto', 100, 1),
 (0.2, 'poly', 3, 'auto', 100, 1),
 (0.2, 'poly', 4, 'auto', 100, 1),
 (0.2, 'rbf', 1, 'scale', 0, 1),
 (0.2, 'rbf', 1, 'auto', 0, 1),
 (0.2, 'sigmoid', 1, 'scale', 0, 1),
 (0.2, 'sigmoid', 1, 'scale', 1, 1),
 (0.

In [357]:
my_combos = []
for c in c_options:
    for p in probability_options:
        for k in kernal_options:
            if k == "linear":
                my_combos += [(c, k, 1, 'auto', 0, p)]
            else:
                for g in gamma_options:
                    if k == "rbf":
                        my_combos += [(c, k, 1, g, 0, p)]
                    else:
                        for co in coef0_options:
                            if k == "sigmoid":
                                my_combos += [(c, k, 1, g, co, p)]
                            else:
                                for d in degree_options:
                                    my_combos += [(c, k, d, g, co, p)]
                                    
best_5_combos = []
random_combos = random.shuffle(my_combos)


                

In [359]:
my_combos

[(4, 'poly', 2, 'auto', 10, 1),
 (1, 'poly', 3, 'scale', 1, 1),
 (8, 'linear', 1, 'auto', 0, 0),
 (4, 'poly', 4, 'auto', 10, 0),
 (8, 'poly', 3, 'scale', 0, 1),
 (8, 'poly', 3, 'auto', 1, 1),
 (8, 'poly', 3, 'scale', 0, 0),
 (4, 'poly', 3, 'auto', 1, 1),
 (0.2, 'poly', 4, 'auto', 1, 0),
 (4, 'sigmoid', 1, 'auto', 100, 1),
 (1, 'poly', 3, 'scale', 10, 1),
 (1, 'poly', 4, 'auto', 1, 0),
 (4, 'poly', 4, 'auto', 10, 1),
 (0.2, 'poly', 4, 'auto', 100, 1),
 (4, 'rbf', 1, 'auto', 0, 1),
 (0.2, 'poly', 2, 'auto', 100, 1),
 (1, 'poly', 4, 'auto', 1, 1),
 (0.2, 'linear', 1, 'auto', 0, 0),
 (1, 'poly', 3, 'scale', 10, 0),
 (1, 'sigmoid', 1, 'auto', 10, 0),
 (1, 'sigmoid', 1, 'scale', 100, 1),
 (8, 'poly', 4, 'scale', 1, 1),
 (1, 'linear', 1, 'auto', 0, 0),
 (0.2, 'poly', 2, 'auto', 100, 0),
 (8, 'sigmoid', 1, 'scale', 0, 0),
 (4, 'sigmoid', 1, 'scale', 100, 0),
 (0.2, 'linear', 1, 'auto', 0, 1),
 (1, 'poly', 2, 'scale', 1, 1),
 (1, 'sigmoid', 1, 'auto', 100, 0),
 (4, 'linear', 1, 'auto', 0, 0),
 

In [349]:
def update_best_score(c, best_combo, best_score):
    SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
    SVM.fit(Train_X_Tfidf,Train_Y)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    my_f1_score = f1_score(predictions_SVM, Test_Y)*100
    print("SVM Accuracy Score -> ",my_f1_score)
    if (my_f1_score > best_score):
        best_score = my_f1_score
        best_combo = c
    return best_combo, best_score

In [387]:
Tfidf_vect = TfidfVectorizer(max_features=None)
Tfidf_vect.fit(df['3_in_one_text'])
cross_val_Tfidf = Tfidf_vect.transform(df['3_in_one_text'])

In [391]:
cross_val_Tfidf.toarray()

array([[0.        , 0.05678301, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05183343, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [398]:
from sklearn.metrics import SCORERS

In [399]:
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [401]:
best_combo = {}

for c in my_combos[:100]:
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
    scores = cross_val_score(SVM, cross_val_Tfidf, df['would_shortlist'], cv=10, scoring='recall')
    # predict the labels on validation dataset
    #predictions_SVM = SVM.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    #my_recall_score = recall_score(predictions_SVM, Test_Y)*100
    print("SVM Accuracy Score -> ",scores.mean())
    best_combo[scores.mean()] = (c, scores.std())

SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.3833333333333333
SVM Accuracy Score ->  0.46619047619047616
SVM Accuracy Score ->  0.5647619047619048
SVM Accuracy Score ->  0.13238095238095238
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.13238095238095238
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  1.0
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.4523809523809524
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.5647619047619048
SVM Accuracy Score ->  0.3833333333333333
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  1.0
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.6752380952380952
SVM Accuracy Score ->  0.4523809523809524
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.30047619047619045
SVM Accuracy Score ->  0.5514285714285714
SVM Accuracy Score ->  1.0
SVM Accuracy Score ->  0.5161904761904761
SVM Accuracy Score ->  0.2
SVM Accuracy Score ->  0.6752380952380952
SVM Accuracy Score ->  0.45285714285714285
SVM Accuracy S

In [ ]:
best_combo = {}

for c in my_combos[:100]:
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
    SVM.fit(Train_X_Tfidf,Train_Y)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    my_recall_score = recall_score(predictions_SVM, Test_Y)*100
    print("SVM Accuracy Score -> ",my_recall_score)
    best_combo[my_recall_score] = c

In [376]:
best_combo[100]

(8, 'poly', 4, 'scale', 0, 1)

In [379]:
c = best_combo[100]
SVM = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Train_X_Tfidf)
# Use accuracy_score function to get the accuracy
my_recall_score = recall_score(predictions_SVM, Train_Y)*100
print("SVM Accuracy Score -> ",my_recall_score)

SVM Accuracy Score ->  100.0


In [385]:
c = best_combo[100]
SVM_best = svm.SVC(C=c[0], kernel=c[1], degree=c[2], gamma=c[3], coef0 = c[4], probability = c[5], class_weight = 'balanced')

In [384]:
sum(predictions_SVM)

132

In [ ]:
list(best_combo.keys).sort()

In [302]:
predictions_NB

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

In [265]:
word_count_matrix.toarray() # represents occurance of each word

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)